In [11]:
import os
import pandas as pd
import matplotlib.pyplot as plt
import plotly.express as px
import panel as pn
from panel.interact import interact
from panel import widgets
pn.extension('plotly')
import hvplot.pandas
from pathlib import Path
import datetime as dt 
import alpaca_trade_api as tradeapi
from dotenv import load_dotenv

ModuleNotFoundError: No module named 'alpaca_trade_api'

In [7]:
load_dotenv()

True

In [8]:
# Import the CSVs to Pandas DataFrames
abb_path = Path("../data/cleandata/Robintrack_Data/ABB_csv_clean.csv")
amd_path = Path("../data/cleandata/Robintrack_Data/AMD_csv_clean.csv")
riot_path = Path("../data/cleandata/Robintrack_Data/RIOT_csv_clean.csv")
tsla_path = Path("../data/cleandata/Robintrack_Data/TSLA_csv_clean.csv")

abb_data = pd.read_csv(abb_path, infer_datetime_format=True, parse_dates=True)
amd_data = pd.read_csv(amd_path, infer_datetime_format=True, parse_dates=True)
riot_data = pd.read_csv(riot_path, infer_datetime_format=True, parse_dates=True)
tsla_data = pd.read_csv(tsla_path, infer_datetime_format=True, parse_dates=True)

In [10]:
# Set Alpaca API key and secret
alpaca_api_key = os.getenv("GGC_ALPACA_API_KEY")
alpaca_secret_key = os.getenv("GGC_ALPACA_SECRET_KEY")

# Create the Alpaca API object
alpaca = tradeapi.REST(
    alpaca_api_key,
    alpaca_secret_key,
    api_version="v2"
)

NameError: name 'tradeapi' is not defined